In [29]:
include("Create_Sample.jl")
include("MIP_SPLIT.jl")
include("Split.jl")
include("GA.jl")
include("Initial.jl")
include("Mutation.jl")
include("Crossover.jl")
include("Neighborhood.jl")
include("Neighborhood_intra.jl")

Ni7 (generic function with 1 method)

In [211]:
# n = 50
# length_of_area = 10
# tspeed = 40
# TT = Creat_Sample(n, length_of_area, tspeed)
# chrm = shuffle!([i for i = 1:n])
# demands = rand(1:10, n)
T = Read_TSPLIB_instance(:eil51, 1)
n = size(T)[1]-2
demands = ones(Int, n)
print("done")

done

In [ ]:
instances = [:eil51, :berlin52]
Ms = [2,3,5,7]

for instance in instances
    for K in Ms
# K = 2
        T = Read_TSPLIB_instance(instance, 1)
        n = size(T)[1]-2
        demands = ones(Int, n)
        W = 150
        h = 1.0
        popsize = (10,50)
        k_tournament = 3
        num_iter = 20000
        Mutation_Chance = 0.1
        num_runs = 10
        avg = 0.0
        best = Inf
        worst = 0.0
        t1 = time() 
        P = Chromosome[]
        for i=1:num_runs
            P = Perform_Genetic_Algorithm(T, demands,K, W, h, popsize, k_tournament, num_iter, Mutation_Chance);
            avg += P[1].fitness
            if P[1].fitness < best
                best = P[1].fitness
            end
            if P[1].fitness > worst
                worst = P[1].fitness
            end
        end
        t2 = time()
        println("Results for ", instance, " ,m=", K)
        println("Best: ", round(best, digits = 1), "  Average: ", round(avg/num_runs, digits = 1), 
            "  Worst: ", round(worst, digits = 1), " , run time= ", round((t2-t1)/num_runs, digits=0))
    end
end

The best objective achieved in 22144 generations is: 225.2333319859696 and it took 6.190999984741211 seconds.
The best objective achieved in 22090 generations is: 225.2333319859696 and it took 4.491999864578247 seconds.
The best objective achieved in 22302 generations is: 225.2333319859696 and it took 4.209000110626221 seconds.
The best objective achieved in 38251 generations is: 225.2333319859696 and it took 7.246999979019165 seconds.
The best objective achieved in 30207 generations is: 226.3840426890465 and it took 5.713000059127808 seconds.
The best objective achieved in 20866 generations is: 225.2333319859696 and it took 4.128000020980835 seconds.


In [119]:
for tour in P[1].tours
    t1 = copy(tour.Sequence);
    pushfirst!(t1, 0)
    push!(t1, n+1)

    z1 = 0.0
    for i=1:length(t1)-1
        z1 += T[t1[i]+1, t1[i+1]+1]
    end

    println(z1, " ,  ", tour.cost)
end

LoadError: BoundsError: attempt to access 0-element Vector{Chromosome} at index [1]

In [215]:
print(:berlin52)

berlin52

In [10]:
maximum([P[1].tours[i].cost for i=1:length(P[1].tours)])

2693.3316949626014

In [8]:
for i in P[1].genes
    print(i," ")
end

39 37 47 23 24 11 50 10 25 46 18 38 36 4 42 32 9 8 7 40 13 51 12 26 27 44 29 41 1 6 20 16 2 30 17 48 21 34 35 31 43 45 3 14 5 33 15 49 19 22 28 

In [10]:
for tour in P[1].tours
    cost = 0.0
    seq = copy(tour.Sequence)
    pushfirst!(seq, 0)
    push!(seq, n+1)
    for i=1:length(seq)-1
        cost += T[seq[i]+1,seq[i+1]+1]
    end
    println(cost)
end

2557.7022429650438
2516.602068167163
2507.8846234183943
2594.0315040190276
2609.487351461127


In [9]:
T[1,51]+T[51,46]+T[46,52]

2294.1478554084824

In [29]:
function Find_Closeness(TT::Matrix{Float64}, h::Float64)
    n_nodes = size(TT)[1]-2
    num = Int(ceil(h*n_nodes))
    ClosenessT = zeros(Int, n_nodes,num)
    for i=2:n_nodes+1
        a = copy(TT[i,2:n_nodes+1])
        b = sortperm(a)
        ClosenessT[i-1,:] = b[2:num+1]
    end
    return ClosenessT
end

Find_Closeness (generic function with 1 method)

In [21]:
Find_Closeness(T, 0.2)

52×11 Matrix{Int64}:
  6  41  29  20  16  22  30  19  17  49  28
 17  18  30  16  44  21  31  40  48  20   7
  5  14   4  24  23  47  37  39  36  38  45
 14  23   5  47  37  39  36   3  38  24  33
  4  14  23   3  47  37  39  36  24  38  45
  1  41  20  29  16  22  30  19  17  49  21
 40  18   9   8  44   2  31  42  17  48  38
  9   7  40  18  44  42   2  31  48  39  38
  8   7  40  18  42  44  31   2  14  39  38
 50  11  51  26  27  12  25  13  24   3  46
 27  50  24  26   3  25   5   4  10  23  14
 26  13  25  51  27  46  10  11  50  24  45
 12  51  46  26  25  27  10  11  50  24  45
  ⋮                   ⋮                   ⋮
  6   1  20  29  16  22  30  19  17  49  21
 14   3   5   4  37  39  23   9  47  38  36
 33  34  35  36  45  15  38  48  39  47  37
 18  31  40   7  48   2  17  21  35  38  34
 47  43  36  23  15  33  37   4  24  39   5
 25  26  12  27  13  11  28  15  45  24  51
 23   4  37  36   5  14  39  38  45  33   3
 31  35  34  21  33  38  39  43  36  17  37
 19  22  15

In [38]:
T[]

3

In [12]:
function test(a::Int)
    a += 1
    return a
end


test (generic function with 1 method)

In [13]:
x = 5

test(x)

6

In [22]:
x = []
a = [1,2]
b = [4,7]
c = [100, 1000, 10000]
x = vcat(x,a)
x = vcat(x,b)
x = vcat(x,c)


7-element Vector{Any}:
     1
     2
     4
     7
   100
  1000
 10000

In [83]:
a = [1,2,3,4,5]
k = 3
insert!(a, k, 100)
insert!(a, k+1, 1000)

7-element Vector{Int64}:
    1
    2
  100
 1000
    3
    4
    5

In [87]:
sort!(sample(1:10, 3, replace=false))

3-element Vector{Int64}:
  3
  8
 10

In [97]:
a

7-element Vector{Int64}:
    1
    2
  100
 1000
    3
    4
    5

In [103]:
a[2:5] = shuffle!(a[2:5])

4-element Vector{Int64}:
    3
 1000
    2
  100

In [157]:
T

53×53 Matrix{Float64}:
    0.0      666.108   281.114   395.601   …   789.383  1220.46      0.0
  666.108      0.0     649.327  1047.09       1421.56   1716.05    666.108
  281.114    649.327     0.0     603.511       995.314  1483.59    281.114
  395.601   1047.09    603.511     0.0         397.02    908.639   395.601
  291.204    945.145   508.945   104.403       499.925   984.441   291.204
  326.267    978.085   542.517    69.6419  …   464.57    954.895   326.267
  640.8       45.0     610.574  1026.36       1405.08   1715.07    640.8
  426.878    956.151   308.058   525.0         860.145  1430.47    426.878
  600.187   1134.96    485.644   611.003       883.233  1486.78    600.187
  561.471   1132.98    487.263   533.901       800.078  1404.04    561.471
 1040.97    1638.79   1266.69    663.193   …   285.044   398.56   1040.97
  655.019   1258.59    891.361   294.364       188.215   618.567   655.019
  975.0     1440.08   1247.76    711.073       539.676   278.657   975.0
    ⋮    

In [190]:
function find_tsp_tour1(Ct::Matrix{Float64})
    scale_factor = 1000
    dist_mtx = round.(Int, Ct .* scale_factor)

    # tsp_tour, tsp_tour_len = Concorde.solve_tsp(dist_mtx)
    tsp_tour, tsp_tour_len = Concorde.solve_tsp(dist_mtx)

    @assert tsp_tour[1] == 1

    return tsp_tour[2:length(tsp_tour)].-1, tsp_tour_len/1000
end

find_tsp_tour1 (generic function with 1 method)

In [199]:
using Concorde
tour, _ = find_tsp_tour1(T[1:52, 1:52])

51-element Vector{Int64}:
 21
 30
 17
  2
 16
 20
 41
  6
  1
 29
 22
 19
 49
  ⋮
 10
 50
 32
 42
  9
  8
  7
 40
 18
 44
 31
 48

In [200]:
length(demands)

51

In [194]:
pushfirst!(tour, 0)
push!(tour, 52)
z1 = 0.0
for i=1:length(tour)-1
    z1 += T[tour[i]+1, tour[i+1]+1]
end 
z1

7544.36590190409

In [163]:
for tour in P[1].tours
    for i in tour.Sequence 
        print(i,"  ")
    end
    println()
end

3  50  10  32  42  9  8  7  40  18  44  
38  39  36  37  47  23  4  14  5  24  11  27  26  12  51  13  46  25  
21  31  48  35  34  33  43  45  15  28  29  6  1  41  20  16  2  17  30  22  19  49  


In [166]:
tsp = readTSPLIB(:berlin52)

TSP("berlin52", 52, "EUC_2D", [0.0 666.0 … 789.0 1220.0; 666.0 0.0 … 1422.0 1716.0; … ; 789.0 1422.0 … 0.0 625.0; 1220.0 1716.0 … 625.0 0.0], [565.0 575.0; 25.0 185.0; … ; 1340.0 725.0; 1740.0 245.0], false, TSPLIB.var"#fFit#19"{Matrix{Float64}, Int64}([0.0 666.0 … 789.0 1220.0; 666.0 0.0 … 1422.0 1716.0; … ; 789.0 1422.0 … 0.0 625.0; 1220.0 1716.0 … 625.0 0.0], 52), TSPLIB.var"#pFit#20"{Matrix{Float64}, Int64}([0.0 666.0 … 789.0 1220.0; 666.0 0.0 … 1422.0 1716.0; … ; 789.0 1422.0 … 0.0 625.0; 1220.0 1716.0 … 625.0 0.0], 52), 7542.0)

In [169]:
tsp.nodes

52×2 Matrix{Float64}:
  565.0   575.0
   25.0   185.0
  345.0   750.0
  945.0   685.0
  845.0   655.0
  880.0   660.0
   25.0   230.0
  525.0  1000.0
  580.0  1175.0
  650.0  1130.0
 1605.0   620.0
 1220.0   580.0
 1465.0   200.0
    ⋮    
  475.0   960.0
   95.0   260.0
  875.0   920.0
  700.0   500.0
  555.0   815.0
  830.0   485.0
 1170.0    65.0
  830.0   610.0
  605.0   625.0
  595.0   360.0
 1340.0   725.0
 1740.0   245.0

In [182]:
i1 = 2
i2 = 5

euclidean(tsp.nodes[i1,:], tsp.nodes[i2,:])

945.1454914456292

In [183]:
T[2,5]

945.1454914456292

In [174]:
allNodes = tsp.nodes
tspeed = 1
num_of_nodes = size(allNodes)[1] - 1
T = Matrix{Float64}(undef, num_of_nodes + 2, num_of_nodes + 2)
depot = allNodes[1, :]
Nodes = allNodes[2:num_of_nodes+1, :]

T[1, 1] = 0.0
T[num_of_nodes+2, num_of_nodes+2] = 0.0
T[1, num_of_nodes+2] = 0.0
T[num_of_nodes+2, 1] = 0.0
@inbounds for i in 1:num_of_nodes
    T[1, i+1] = euclidean(depot, Nodes[i, :]) / tspeed
    T[i+1, 1] = T[1, i+1]
    T[num_of_nodes+2, i+1] = T[1, i+1]
    T[i+1, num_of_nodes+2] = T[1, i+1]
    @inbounds for j in 1:num_of_nodes
        T[i+1, j+1] = euclidean(Nodes[i, :], Nodes[j, :]) / tspeed
        T[j+1, i+1] = T[i+1, j+1]
    end
end

In [176]:
depot

2-element Vector{Float64}:
 565.0
 575.0

In [178]:
Nodes[1,:]

2-element Vector{Float64}:
  25.0
 185.0

In [179]:
euclidean(depot, Nodes[1,:])

666.1080993352356